# Lynx



You'd need a GPU with atleast 32GB RAM in order to load the smallest Lynx model i.e. 8B FP32.

You'd need atleast 4 GPUs with 40GB RAM each to load the bigger Lynx model i.e. 70B FP32.

Install dependencies

In [ ]:
%pip install transformers
%pip install datasets --quiet

Load model

In [ ]:
from vllm import LLM, SamplingParams

model_name_70b = 'PatronusAI/Llama-3-Patronus-Lynx-70B-Instruct'

model_70b = LLM(model_name_70b, tensor_parallel_size=4, gpu_memory_utilization=0.98)

INFO 12-16 05:42:19 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-16 05:42:20 config.py:1020] Defaulting to use mp for distributed inference
INFO 12-16 05:42:20 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='PatronusAI/Llama-3-Patronus-Lynx-70B-Instruct', speculative_config=None, tokenizer='PatronusAI/Llama-3-Patronus-Lynx-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, c

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


INFO 12-16 05:51:31 model_runner.py:1077] Loading model weights took 32.8599 GB
(VllmWorkerProcess pid=3495) (VllmWorkerProcess pid=3497) INFO 12-16 05:51:31 model_runner.py:1077] Loading model weights took 32.8599 GB
INFO 12-16 05:51:31 model_runner.py:1077] Loading model weights took 32.8599 GB
(VllmWorkerProcess pid=3496) INFO 12-16 05:51:31 model_runner.py:1077] Loading model weights took 32.8599 GB
(VllmWorkerProcess pid=3496) (VllmWorkerProcess pid=3497) (VllmWorkerProcess pid=3495) INFO 12-16 05:51:35 worker.py:232] Memory profiling results: total_gpu_memory=39.38GiB initial_memory_usage=34.19GiB peak_torch_memory=33.58GiB memory_usage_post_profile=35.11GiB non_torch_memory=2.24GiB kv_cache_size=2.78GiB gpu_memory_utilization=0.98
INFO 12-16 05:51:35 worker.py:232] Memory profiling results: total_gpu_memory=39.38GiB initial_memory_usage=34.33GiB peak_torch_memory=33.58GiB memory_usage_post_profile=35.39GiB non_torch_memory=2.52GiB kv_cache_size=2.50GiB gpu_memory_utilization=0.9

### Halubench

In [ ]:
from datasets import load_dataset, Dataset

ds = load_dataset("PatronusAI/HaluBench")
data = ds["test"].to_pandas()

# store sources for subsets
sources = data.source_ds.unique().tolist()
sources.remove('RAGTruth')
sources.remove('halueval')

In [ ]:
# Lynx prompt template
def generate_prompt(question, document, answer):
    prompt = f"""
    Given the following QUESTION, DOCUMENT and ANSWER you must analyze the provided answer and determine whether it is faithful to the contents of the DOCUMENT. The ANSWER must not offer new information beyond the context provided in the DOCUMENT. The ANSWER also must not contradict information provided in the DOCUMENT. Output your final verdict by strictly following this format: "PASS" if the answer is faithful to the DOCUMENT and "FAIL" if the answer is not faithful to the DOCUMENT. Show your reasoning.
    --
    QUESTION (THIS DOES NOT COUNT AS BACKGROUND INFORMATION):
    {question}
    --
    DOCUMENT:
    {document}
    --
    ANSWER:
    {answer}
    --
    Your output should be in JSON FORMAT with the keys "REASONING" and "SCORE":
    {{"REASONING": "<your reasoning as bullet points>", "SCORE": "<your final score>"}}
    """
    return prompt

In [ ]:
import os, json

def save_to_json(filename, reasonings, scores, source):
    # check if json exists
    if os.path.exists(f'{filename}.json'):
        with open(f'{filename}.json', 'r') as f:
            results_dict = json.load(f)
    else:
        results_dict = {}

    # add the results under source key
    results_dict[source] = [{'reasoning': reasoning, 'score': score} for reasoning, score in zip(reasonings, scores)]

    # write the updated dictionary
    with open(f'{filename}.json', 'w') as f:
        json.dump(results_dict, f, indent=2)

In [ ]:
from tqdm import tqdm
import re

# extract reasoning and score from LLM response
def postprocess(results):
  reasonings = []
  scores = []
  none_score = 0

  reasoning_pattern = r'"REASONING":\s*\[(.*?)\]'
  score_pattern = r'"SCORE":\s*(\w+)'

  for response in results:
      text = response.outputs[0].text
      reasoning_match = re.search(reasoning_pattern, text, re.DOTALL)
      reasoning = [r.strip("'") for r in reasoning_match.group(1).split("', '")] if reasoning_match else None
      reasonings.append(reasoning)

      score_match = re.search(score_pattern, text)
      if score_match:
          score = score_match.group(1)
          if score == "PASS":
              score_num = 1
          elif score == "FAIL":
              score_num = 0
          else:
              score_num = None
      else:
          none_score+=1
          score_num = None
      scores.append(score_num)

  print("Responses without score: ", none_score)
  return reasonings, scores

def parse_output(response):
    reasoning_pattern = r'"REASONING":\s*\[(.*?)\]'
    score_pattern = r'"SCORE":\s*(\w+)'

    text = response.outputs[0].text
    reasoning_match = re.search(reasoning_pattern, text, re.DOTALL)
    reasoning = [r.strip("'") for r in reasoning_match.group(1).split("', '")] if reasoning_match else None

    score_match = re.search(score_pattern, text)
    if score_match:
        score = score_match.group(1)
        if score == "PASS":
            score_num = 1
        elif score == "FAIL":
            score_num = 0
        else:
            score_num = None
    else:
        score_num = None

    return reasoning, score_num

In [ ]:
def run_metrics(data, sources, model, filename):
    results = {}
    max_retries = 10
    for source in sources:
        # extract subset
        subset = data[data.source_ds == source]

        prompts = [generate_prompt(question, document, answer) for question, document, answer in zip(subset['question'], subset['passage'], subset['answer'])]
        params = SamplingParams(max_tokens=1024)

        responses = model.generate(prompts, params, use_tqdm=True)

        to_retry_inputs = []
        to_retry_indices = []
        for i, resp in enumerate(responses):
            feedback, score = parse_output(resp)
            if feedback is None:
                to_retry_inputs.append(prompts[i])
                to_retry_indices.append(i)

        # Retry logic with progress bar
        retries = 0
        while to_retry_inputs and retries < max_retries:
            retries += 1
            print(f"Retrying failed batches: Attempt {retries}/{max_retries}")
            retry_outputs = model.generate(to_retry_inputs, params, use_tqdm=True)

            new_to_retry_inputs = []
            new_to_retry_indices = []
            for idx, (retry_idx, output) in enumerate(zip(to_retry_indices, retry_outputs)):
                feedback, score = parse_output(output)
                if feedback is None:  # Still failing
                    new_to_retry_inputs.append(to_retry_inputs[idx])
                    new_to_retry_indices.append(to_retry_indices[idx])
                else:
                    responses[retry_idx] = output  # Update with successful retry

            to_retry_inputs = new_to_retry_inputs
            to_retry_indices = new_to_retry_indices

        results[source] = responses
        # postprocess to again extract reasoning and score
        reasonings, scores = postprocess(responses)
        # save results to json
        save_to_json(filename, reasonings, scores, source)

        print("Completed evaluation on {0} dataset. Length of feedback: {1} and scores: {2}".format(source, len(reasonings), len(scores)))
    return results

In [ ]:
filename = 'lynx_70b'

results = run_metrics(data, sources, model_70b, filename)

Processed prompts:   0%|                                        | 2/1000 [00:03<27:40,  1.66s/it, est. speed input: 228.07 toks/s, output: 0.50 toks/s]

WARNING 12-09 07:18:29 scheduler.py:1481] Sequence group 86 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts:   9%|███▎                                 | 89/1000 [01:15<14:54,  1.02it/s, est. speed input: 554.20 toks/s, output: 405.79 toks/s]

WARNING 12-09 07:19:41 scheduler.py:1481] Sequence group 150 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


Processed prompts:  16%|█████▋                              | 159/1000 [02:15<13:55,  1.01it/s, est. speed input: 537.97 toks/s, output: 453.09 toks/s]

...

Processed prompts:  67%|████████████████████████▉            | 442/657 [35:45<22:21,  6.24s/it, est. speed input: 839.57 toks/s, output: 104.83 toks/s]

WARNING 12-09 11:09:28 scheduler.py:1481] Sequence group 6904 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=2651


Processed prompts: 100%|█████████████████████████████████████| 657/657 [54:50<00:00,  5.01s/it, est. speed input: 817.46 toks/s, output: 104.42 toks/s]


Retrying failed batches: Attempt 2/10


Processed prompts:  36%|█████████████▎                       | 161/448 [13:53<18:05,  3.78s/it, est. speed input: 808.82 toks/s, output: 100.56 toks/s]

### ELI5

In [ ]:
def run_metrics(data, model, filename):
    results = {}
    max_retries = 10
    source = 'eli5'
    prompts = [generate_prompt(question, document, answer) for question, document, answer in zip(data['user_input'], data['reference'], data['response'])]
    params = SamplingParams(max_tokens=1024)

    responses = model.generate(prompts, params, use_tqdm=True)

    to_retry_inputs = []
    to_retry_indices = []
    for i, resp in enumerate(responses):
        feedback, score = parse_output(resp)
        if feedback is None:
            to_retry_inputs.append(prompts[i])
            to_retry_indices.append(i)

    # Retry logic with progress bar
    retries = 0
    while to_retry_inputs and retries < max_retries:
        retries += 1
        print(f"Retrying failed batches: Attempt {retries}/{max_retries}")
        retry_outputs = model.generate(to_retry_inputs, params, use_tqdm=True)

        new_to_retry_inputs = []
        new_to_retry_indices = []
        for idx, (retry_idx, output) in enumerate(zip(to_retry_indices, retry_outputs)):
            feedback, score = parse_output(output)
            if feedback is None:  # Still failing
                new_to_retry_inputs.append(to_retry_inputs[idx])
                new_to_retry_indices.append(to_retry_indices[idx])
            else:
                responses[retry_idx] = output  # Update with successful retry

        to_retry_inputs = new_to_retry_inputs
        to_retry_indices = new_to_retry_indices

    results[source] = responses
    # postprocess to extract reasoning and score
    reasonings, scores = postprocess(responses)
    # save results to json
    save_to_json(filename, reasonings, scores, source)

    print("Completed evaluation on {0} dataset. Length of feedback: {1} and scores: {2}".format(source, len(reasonings), len(scores)))
    return results

In [ ]:
from datasets import load_dataset

ds = load_dataset("explodinggradients/ELI5")
data = ds["train"].to_pandas()

filename = 'lynx_70b'

results = run_metrics(data, model_70b, filename)

Processed prompts:   3%|██                                                                    | 3/100 [00:05<02:18,  1.43s/it, est. speed input: 219.47 toks/s, output: 0.56 toks/s]

WARNING 12-16 05:53:30 scheduler.py:1481] Sequence group 58 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████████████████████████████████████████████████████████████| 100/100 [01:38<00:00,  1.02it/s, est. speed input: 413.89 toks/s, output: 544.98 toks/s]


Retrying failed batches: Attempt 1/10


Processed prompts: 100%|████████████████████████████████████████████████████████████████████| 38/38 [00:43<00:00,  1.16s/it, est. speed input: 351.71 toks/s, output: 494.67 toks/s]


...

Retrying failed batches: Attempt 3/10


Processed prompts: 100%|███████████████████████████████████████████████████████████████████████| 6/6 [00:36<00:00,  6.16s/it, est. speed input: 69.29 toks/s, output: 123.36 toks/s]


Retrying failed batches: Attempt 4/10


Processed prompts: 100%|████████████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.31s/it, est. speed input: 45.13 toks/s, output: 39.72 toks/s]

Responses without score:  0
Completed evaluation on eli5 dataset. Length of feedback: 100 and scores: 100
